In [12]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import timeit
from glob import glob

<h1>Load training, validation, testing set from your preprocessed files</h1>

In [22]:
files_path = 'dogs-vs-cats-preprocessed/'

train_files_path = os.path.join(files_path, 'train*')
train_files = sorted(glob(train_files_path))
valid_files_path = os.path.join(files_path, 'valid')

In [28]:
img_size = 227
X_train = np.empty((0, img_size, img_size, 3))
Y_train = np.empty((0,2));
for f in train_files:
    train_image, train_labels = pickle.load(open(f, 'rb'))
    X_train = np.concatenate((X_train, train_image), axis = 0)
    Y_train = np.concatenate((Y_train, train_labels), axis = 0)

In [29]:
X_valid, Y_valid = pickle.load(open(valid_files_path, 'rb'))
print(X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape)

(24500, 227, 227, 3) (24500, 2) (500, 227, 227, 3) (500, 2)


<h1>Define hyperparameter</h1>

In [67]:
batch_size = 128
num_epochs = 10
lr = 0.01
num_conv_layers = 6
N = X_train.shape[0]
def mini_batch(features,labels,batch_size):
    for start_idx in range(0, N - batch_size + 1, batch_size):
        indices = np.arange(features.shape[0])
        np.random.shuffle(indices)
        excerpt = indices[start_idx:start_idx + batch_size]
        yield features[excerpt], labels[excerpt]


<h1>Placeholder</h1>

In [31]:
tf.reset_default_graph()

<h1>AlexNet</h1>

In [68]:
def initalize_weights(shape):
    return tf.Variable(tf.truncated_normal(
        shape = shape, 
        mean = 0, 
        stddev = 0.1))
def initialize_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))
class AlexNet:
    def __init__(self,learning_rate = lr, batch_size = batch_size,num_epochs = num_epochs):
        self.learning_rate = lr
        self.batch_size = batch_size
        self.epochs = num_epochs
    
    def model(self,x):
        with tf.name_scope("conv1") as scope:
            conv1_W = initalize_weights((11,11,3,96))
            conv1_b = initialize_bias([96])
            conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 4, 4, 1], padding='SAME') + conv1_b
            conv1 = tf.nn.relu(conv1)

            mean, var = tf.nn.moments(conv1, [0, 1, 2])
            conv1_bn = tf.nn.batch_normalization(conv1, mean, var, 0, 1, 0)
            
            conv1_maxpool = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name = scope)
        
        with tf.name_scope("conv2") as scope:
            conv2_W = initalize_weights((5,5,96,256))
            conv2_b = initialize_bias([256])
            conv2   = tf.nn.conv2d(conv1_maxpool, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
            conv2 = tf.nn.relu(conv2)

            mean, var = tf.nn.moments(conv2, [0, 1, 2])
            conv2_bn = tf.nn.batch_normalization(conv2, mean, var, 0, 1, 0)
            
            conv2_maxpool = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name = scope)
        
        with tf.name_scope("conv3") as scope:
            conv3_W = initalize_weights((3,3,256,512))
            conv3_b = initialize_bias([512])
            conv3   = tf.nn.conv2d(conv2_maxpool, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
            conv3 = tf.nn.relu(conv3)

            mean, var = tf.nn.moments(conv3, [0, 1, 2])
            conv3_bn = tf.nn.batch_normalization(conv3, mean, var, 0, 1, 0)
            
            conv3 = tf.nn.max_pool(conv3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name = scope)
            
        with tf.name_scope("conv4") as scope:
            conv4_W = initalize_weights((3,3,512,1024))
            conv4_b = initialize_bias([1024])
            conv4   = tf.nn.conv2d(conv3, conv4_W, strides=[1, 1, 1, 1], padding='SAME') + conv4_b
            conv4 = tf.nn.relu(conv4)

            mean, var = tf.nn.moments(conv4, [0, 1, 2])
            conv4_bn = tf.nn.batch_normalization(conv4, mean, var, 0, 1, 0)
            
            conv4 = tf.nn.max_pool(conv4, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name = scope)
            
        with tf.name_scope("conv5") as scope:
            conv5_W = initalize_weights((3,3,1024,512))
            conv5_b = initialize_bias([512])
            conv5   = tf.nn.conv2d(conv4, conv5_W, strides=[1, 1, 1, 1], padding='SAME') + conv5_b
            conv5 = tf.nn.relu(conv5)

            mean, var = tf.nn.moments(conv5, [0, 1, 2])
            conv5_bn = tf.nn.batch_normalization(conv5, mean, var, 0, 1, 0)
            
            conv5 = tf.nn.max_pool(conv4, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name = scope)
        
        with tf.name_scope('FC1') as scope:
            pool5_flat = tf.contrib.layers.flatten(conv5)
            fc1 = tf.layers.dense(pool5_flat, units=1024, activation=tf.nn.relu, name=scope)
            # fc6_dropout = tf.layers.dropout(fc6, 0.5)

        with tf.name_scope('FC2') as scope:
            fc2 = tf.layers.dense(fc1, units=256, activation=tf.nn.relu, name=scope)
            # fc7_dropout = tf.layers.dropout(fc7, 0.5)

        with tf.name_scope('logits') as scope:
            logits = tf.layers.dense(fc2, units=2, activation=tf.nn.softmax, name=scope)    
        return logits
    
    def train(self):
        X = tf.placeholder(tf.float32, shape=[None, 227,227,3])
        Y = tf.placeholder(tf.float32, shape=[None, 2])
        logits = self.model(X)
        loss = tf.nn.l2_loss(tf.square(logits - Y))
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(loss)
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        init = tf.global_variables_initializer()
        
        with tf.Session() as sess:
            sess.run(init)
            for i in range(num_epochs):
                #fetch batch
                for img_batch, labels_batch in mini_batch(X_train,Y_train, self.batch_size):
                    sess.run(optimizer, feed_dict={X:img_batch, Y:labels_batch})
                acc = sess.run(accuracy,feed_dict={X:X_train, Y:Y_train})
                print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
            print("Training finished!") 
            print("Testing Accuracy:", sess.run(accuracy, feed_dict={X:X_valid, Y:Y_valid})) 

In [ ]:
alex_net = AlexNet()
alex_net.train()

<h1>Cost and Optimization</h1>

<h1>Training and validation</h1>
<h2>Train your model only 10 epochs</h2>
<p style="font-size:20px">1. Print out training accuracy and validation accuracy each training epoch</p>
<p style="font-size:20px">2. Print out training time each training epoch</p>
<p style="font-size:20px">3. Your goal is to reach 85% validation accuracy in 10 training epochs. If you reach that, you can perform testing, print out your test accuracy. Plot out the ten images with title that contains the probability of the labeled class.</p>